In [15]:
# -*- coding: utf-8 -*-

from dateutil import parser
import pdfminer
from pdfminer.high_level import extract_text
import glob
import nltk
from string import punctuation
from nltk.corpus import stopwords
import re
from nltk.tag import pos_tag
from names_dataset import NameDataset
nd = NameDataset()
import spacy #numpy needs to be <1.20
nlp = spacy.load("en_core_web_sm")
import os
import pdfplumber
import pandas as pd
import datefinder
from datetime import datetime
from thefuzz import fuzz

C:\Users\rtd91\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [20]:
!pip install thefuzz

You should consider upgrading via the 'c:\users\rtd91\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install datefinder

You should consider upgrading via the 'c:\users\rtd91\anaconda3\python.exe -m pip install --upgrade pip' command.


In [17]:
def read_file(path):
    txt = extract_text(path, codec='utf-8')
    return txt.encode('utf-8').decode('utf-8')

def read_file2(path):
    txt_lst = []
    with pdfplumber.open(path) as pdf:
        for p in range(len(pdf.pages)):
            txt_lst.append(pdf.pages[p].extract_text())
        txt = ' '.join(txt_lst)
    return txt.encode('utf-8').decode('utf-8')

def preprocess_text(txt):
    # verify type of input
    if not isinstance(txt, str):
        raise TypeError('Work with String Only')
    # verify len(text)
    if len(txt) < 1:
        raise ValueError('Work with nonempty String Only')
    #tokenize
    words = nltk.word_tokenize(txt)

    #remove stopwords and punctuations
    stopwords_set = set(stopwords.words('english'))
    simple_strings = [word for word in words if word not in stopwords_set if word not in punctuation]

    #remove unwanted characters
    res = [re.sub(r'[^\w\s]', '', word) for word in simple_strings]
    res1 = [re.sub(r'^abc(.*?)=[A-Z0-9]+(.*)', r'\1\2', word) for word in res]
    res2 = [str(res) for res in res1]

    res3 = [re.sub(r'/^[A-Za-z]+$/', '', res) for res in res2]
    res3 = [res.replace("ï", "i") if "ï" in res else res for res in res3]
    res3 = [re.sub('\d', '', res) for res in res3]
    res3 = [res.encode('ascii', "ignore").decode() for res in res3]

    return list(filter(None, res3)) #remove empty string from list #return tokens_txt

# name -------------------------------------------------------------------------
indian_last_names = ["Acharya", "Agarwal", "Khatri", "Ahuja", "Anand", "Laghari", "Patel",

                     "Reddy", "Bakshi", "Anthony", "Babu", "Arya", "Balakrishnan", "Banerjee", "Burman", "Bhatt",
                     "Basu", "Bedi", "Varma", "Dara", "Dalal", "Chowdhury",
                     "Chabra", "Chadha", "Chakrabarti", "Chawla", "Ahluwalia", "Amin", "Apte", "Datta", "Deol",
                     "Deshpande", "Dewan", "Lal", "Kohli", "Mangal", "Malhotra", "Jha",
                     "Joshi", "Kapadia", "Iyer", "Jain", "Khanna", "Grover", "Kaur", "Kashyap", "Gokhale", "Ghosh",
                     "Garg", "Dhar", "Gandhi", "Ganguly", "Gupta", "Das", "Chopra", "Dhawan",
                     "Dixit", "Dubey", "Haldar", "Kapoor", "Khurana", "Kulkarni", "Madan", "Bajwa", "Bhasin", "Chandra",
                     "Chauhan", "Deshmukh", "Dayal", "Dhillon", "Goswami", "Goel", "Mallick",
                     "Mahajan", "Kumar", "Mani", "Gill", "Mannan", "Biswas", "Batra", "Bawa", "Mehta", "Mukherjee",
                     "Saxena", "Zacharia", "Shah", "Ray", "Rao", "Purohit", "Parekh", "Thakur", "Singh", "Sharma",
                     "Seth", "Sachdev", "Ranganathan", "Puri", "Pandey", "Naidu", "Modi"]

chinese_last_names = ["Li", "Wang", "Zhang", "Liu", "Chen", "Yang", "Zhao", "Huang", "Zhou",

                      "Wu", "Xu", "Sun", "Hu", "Zhu", "Gao", "Lin", "He", "Guo", "Ma", "Luo", "Liang",

                      "Song", "Zheng", "Xie", "Han", "Tang", "Feng", "Yu", "Dong", "Xiao", "Cheng",

                      "Cao", "Yuan", "Deng", "Xu", "Fu", "Shen", "Zeng", "Peng", "Lu", "Su", "Lu", "Jiang", "Cai",
                      "Jia", "Ding", "Wei", "Xue", "Ye", "Yan",

                      "Yu", "Pan", "Du", "Dai", "Xia", "Zhong", "Wang", "Tian", "Ren", "Jiang", "Fan", "Fang", "Shi",
                      "Yao", "Tan", "Sheng", "Zou", "Xiong", "Jin", "Lu", "Hao", "Kong", "Bai", "Cui",

                      "Kang", "Mao", "Qio", "Qin", "Jiang", "Shu", "Shi", "Gu", "Hou", "Shao", "Meng", "Long", "Wan",
                      "Duan", "Zhang", "Qian", "Tang", "Yin", "Li", "Yi", "Chang", "Wu",

                      "Qiao", "He", "Lao", "Gong", "Wen"]

# chinese_last_names = [chinese_last_name.lower() for chinese_last_name in chinese_last_names]
# indian_last_names = [indian_last_name.lower() for indian_last_name in indian_last_names]

us_lnames = nd.get_top_names(n=2000, use_first_names = False, country_alpha2='US')
china_lnames = nd.get_top_names(n=2000, use_first_names = False, country_alpha2='CN')
indian_lnames = nd.get_top_names(n=2000, use_first_names = False, country_alpha2='IN')

ref_names = list(set(us_lnames['US'] + china_lnames['CN'] + chinese_last_names+ indian_lnames['IN']  + indian_last_names))

def get_name(sentence_txt, tokens_txt): #-> list of person names
    temp_person = []
    nltk_tagged = pos_tag(tokens_txt[:11])
    for k in range(10):
        if nltk_tagged[k][1] == 'NNP' and nltk_tagged[k + 1][1] == 'NNP':
            nltk_name = nltk_tagged[k][0] + ' ' + nltk_tagged[k + 1][0]
            temp_person.append(nltk_name)
            if len(temp_person) > 0:# Comment: stop as long as you got the nltk_name
                return temp_person
        if k > 1:
            if (tokens_txt[k].lower() in indian_last_names) or (tokens_txt[k].lower() in chinese_last_names):
                j = k - 1
                return([tokens_txt[j] + " " + tokens_txt[k]]) #Comment: what if kristin (jiating) chen?

    tagged = nlp(sentence_txt[0:200]) #Comment: assume names exist in the top section
    for word in tagged.ents:
        if word.label_ == "PERSON":
            temp_person.append(word.text)
        return temp_person


def get_name_from_ner(sentence_txt):
    temp_person = []
    tagged = nlp(sentence_txt[0:200]) #Comment: assume names exist in the top section
    for word in tagged.ents:
        if word.label_ == "PERSON":
            temp_person.append(word.text)
        return temp_person
    return list(set(temp_person))

def get_name_from_pos(tokens_txt):
    temp_person = []
    nltk_tagged = pos_tag(tokens_txt[:11])
    for k in range(10):
        if nltk_tagged[k][1] == 'NNP' and nltk_tagged[k + 1][1] == 'NNP':
            nltk_name = nltk_tagged[k][0] + ' ' + nltk_tagged[k + 1][0]
            temp_person.append(nltk_name)
            # if len(temp_person) > 0:  # Comment: stop as long as you got the nltk_name
            #     return temp_person
    return list(set(temp_person))

def get_name_from_ref(tokens_txt):
    temp_person = []
    for k in range(10):
        if k >= 1:
            j = k - 1
            if tokens_txt[k].capitalize() in ref_names:
                if len(tokens_txt[j]) < 2 and j != 0:
                    temp_person.append(tokens_txt[j-1] + " " + tokens_txt[j] + " " + tokens_txt[k])
                else:
                    temp_person.append(tokens_txt[j] + " " + tokens_txt[k])
    return(list(set(temp_person)))



# -----------------------------------------------------------------
RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'Schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
    'University'
]

replace_words = ["|","/",".",":","-"]

def extract_education(txt): # Comment: this is not solid
    edu=set()
    p = re.compile('(EDUCATION)?\n?(.*?),\s+(.*?),(.*?)')
    for m in re.finditer(p, txt):

        for word in RESERVED_WORDS:
            if word in m.group(2).lower():
                edu.add(m.group(2))
    return edu

# -----------------------------
def extract_phone_number(txt):
    PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
    phone = re.findall(PHONE_REG, txt)

    if phone:
        number = ''.join(phone[0])

        if txt.find(number) >= 0 and len(number) < 16:
            return number
    return None

# ---------------------------- -
def extract_email(txt):
    EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
    return re.findall(EMAIL_REG, txt)

# -----------------------------------
def extract_phd_degree(txt):
    phd_set = set()
    p = re.compile('(Ph\.D|PhD|PhD\.).*,|(Ph\.D|PhD|PhD\.).*\\n')
    for m in re.finditer(p, txt):
        candidate_txt = m.group().split(',')[0]
        phd_set.add(' '.join(candidate_txt.split()))
    return phd_set

def extract_phd_degree_from_lines(txt):
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    phd_degrees_lst = []
    for index, line in enumerate(lines_txt):
        if re.match('(Ph\.D|PhD|PhD\.)' ,line):
            phd_degrees_lst.append(line.strip())
    if len(phd_degrees_lst)>0:        
        phd_deg = [' '.join(m.split()) for m in phd_degrees_lst][0]
        phd_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",phd_deg)
        phd_deg2 = re.sub("Current|Cumulative|GPA"," ",phd_deg1)
        for punct in replace_words:
            if punct in phd_deg2:
                phd_deg2 = phd_deg2.replace(punct," ")
    else:
        phd_deg2 = None
    return phd_deg2

# ------------------------------------
neglect_words = [
    'MSSQL',
    'MS LAB',
    'MSMQ',
    'MS-DB',
    'MS-Excel',
    'MS-Access',
    'MATLAB',
    'MARY',
    'MACHINE',
    'MAPE',
    'MSE',
    'MARKOV',
    'MAXIMIZATION',
    'MATPLOTLIB',
    'MATPLOT',
    'MATPLOTLIB',
    'MATRICES',
    'MATRIX',
    'MASSACHUSETTS',
    'Mssql',
    'Ms Lab',
    'Msmq',
    'Ms-db',
    'Ms-excel',
    'Ms-access',
    'Matlab',
    'Mary',
    'Machine',
    'Mape',
    'Mse',
    'Markov',
    'Maximization',
    'Matplotlib',
    'Matplot',
    'Matplotlib',
    'Matrices',
    'Matrix',
    'Massachusetts'
    'mssql',
    'ms lab',
    'msmq',
    'ms-db',
    'ms-excel',
    'ms-access',
    'matlab',
    'mary',
    'machine',
    'mape',
    'mse',
    'markov',
    'maximization',
    'matplotlib',
    'matplot',
    'matplotlib',
    'matrices',
    'matrix',
    'massachusetts'
]

def extract_master_degree(txt):
    ms_set = set()
    masters_set = set()
    p = re.compile('((?i:Master)|MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech).*,|((?i:Master)|MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech).*\\n')  # greedy
    for m in re.finditer(p, txt):
        candidate_txt = m.group().split(',')[0]
        for w in candidate_txt.split():
            if w in neglect_words:
                continue
            else:
                ms_set.add(' '.join(candidate_txt.split()))
        if re.search('(?i:Master)', candidate_txt):
            #print(candidate_txt)
            masters_set.add(' '.join(candidate_txt.split()))
    if len(masters_set) > 0:
        return masters_set
    else:
        return ms_set

def is_valid_date(txt):
    return_lst = []
    matches = datefinder.find_dates(txt)
    for match in matches:
        #print(match)
        return_lst.append(match)
    if len(return_lst):
        return True
    else:
        return False

def extract_master_degree_from_lines(txt):
    year ="(\d{4})"
    month = "((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?))"
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    master_degrees_lst = []
    ms_lst = []
    for index, line in enumerate(lines_txt):
        if re.match('(?i:Master)' ,line):
            master_degrees_lst.append(line.strip())
        if re.match('MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech' ,line):
            ms_lst.append(line.strip())
    if len(master_degrees_lst) > 0:
        # cleaned_master_degress_lst = []
        # for d in master_degrees_lst:
        #     cleaned_master_degress_lst.append(' '.join([m for m in d.split() if m != month or not m.isdigit()]))
        mast_deg = [' '.join(m.split()) for m in master_degrees_lst][0]
        mast_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",mast_deg)
        mast_deg2 = re.sub("Current|Cumulative|GPA"," ",mast_deg1)
        for punct in replace_words:
            if punct in mast_deg2:
                mast_deg2 = mast_deg2.replace(punct," ")
        return mast_deg2
    elif len(ms_lst) > 0:
        mast_deg = [' '.join(m.split()) for m in ms_lst][0]
        mast_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",mast_deg)
        mast_deg2 = re.sub("Current|Cumulative|GPA"," ",mast_deg1)
        for punct in replace_words:
            if punct in mast_deg2:
                mast_deg2 = mast_deg2.replace(punct," ")
        print(mast_deg2)
        return mast_deg2
    else:
        mast_deg2 = None
        return mast_deg2
# ---------------------------------
# doesn't match: \nBachelors of Management Studies (concentration in marketing)
def extract_bachelor_degree(txt):
    bachelors_set = set()
    ba_set = set()
    p = re.compile('((?i:Bachelor)|BS|B\.S\.|BA|B\.A\.|B\.S\.E|B\.tech).*,|((?i:Bachelor)|BS|B\.S\.|BA|B\.A\.|B\.S\.E|B\.tech).*\\n')  # greedy
    for m in re.finditer(p, txt):
        candidate_txt = m.group().split(',')[0] #MS Excel MS word...
        #print(candidate_txt)
        ba_set.add(' '.join(candidate_txt.split()))
        if re.search('(?i:Bachelor)', candidate_txt):
            bachelors_set.add(' '.join(candidate_txt.split()))
        if len(bachelors_set) > 0:
            return bachelors_set
        else:
            return ba_set

def extract_bachelor_degree_from_lines(txt):
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    bachelors_set = []
    ba_set = []
    for index, line in enumerate(lines_txt):
        if re.match('(?i:Bachelor)' ,line):
            bachelors_set.append(line.strip())
        if re.match('BS|B\.S\.|BA|B\.A\.|B\.S\.E|B\.tech' ,line):
            ba_set.append(line.strip())

    if len(bachelors_set) > 0:
        bach_deg = [' '.join(m.split()) for m in bachelors_set][0]
        bach_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",bach_deg)
        bach_deg2 = re.sub("Current|Cumulative|GPA"," ",bach_deg1)
        for punct in replace_words:
            if punct in bach_deg2:
                bach_deg2 = bach_deg2.replace(punct," ")
        print(bach_deg2)
        return bach_deg2
    elif len(ba_set) > 0:
        bach_deg = [' '.join(m.split()) for m in ba_set][0]
        bach_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",bach_deg)
        bach_deg2 = re.sub("Current|Cumulative|GPA"," ",bach_deg1)
        for punct in replace_words:
            if punct in bach_deg2:
                bach_deg2 = bach_deg2.replace(punct," ")
        print(bach_deg2)
        return bach_deg2
    else:
        bach_deg2 = None
        return bach_deg2

# -----------------------------------------------------
def extract_phd_school(txt):
    p = re.compile(
        '(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(Ph.D.|PhD|PhD.)')
    for m in re.finditer(p, txt):
        return m.group(0).split("\n")[0]

def extract_ms_school(txt):
    p = re.compile(
        '(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(M.S.|M.S|M.Sc.|MS|Masters|(?i:Master))')
    for m in re.finditer(p, txt):
        return m.group(0).split("\n")[0]


## Using regex for finding university nearest to Bachelor's degree
def extract_bach_school(txt):
    p = re.compile(
        '(?i)(school|college|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(?:(?!(?i)(school|college|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)|(Bachelors|B.tech|Bachelor\'s|B.A.|BS|B.Sc.|B.S|B.S.))[\s\S])*(Bachelors|B.tech|Bachelor\'s|B.A.|BS|B.Sc.|B.S|B.S.)')
    for m in re.finditer(p, txt):
        return m.group(0).split("\n")[0]

# date -------------------------------------------------------------------------





def extract_master_date(txt1):
    lines_txt = [l.strip() for l in txt1.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]

    for index, line in enumerate(lines_txt):
            if re.match('(?i:Master)' ,line):
                masters_date_lst = []
                start_idx = index
                for i, l in enumerate(lines_txt[start_idx:start_idx + 8]):
                    #print(i, l)
                    if datefinder.find_dates(l):
                        matches = datefinder.find_dates(l)
                        for match in matches:
                            #print(match)
                            masters_date_lst.append(match)
                        if len(masters_date_lst) > 0:
                            return [d.strftime("%m/%Y") for d in masters_date_lst]
            elif re.match('MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech' ,line):
                return_set = []
                start_idx = index
                for _, l in enumerate(lines_txt[start_idx:start_idx + 8]):
                    if datefinder.find_dates(l):
                        matches = datefinder.find_dates(l)
                        for match in matches:
                            #print(match)
                            return_set.append(match)
                        return [d.strftime("%m/%Y") for d in return_set]

def extract_bachelor_date(txt1):
    lines_txt = [l.strip() for l in txt1.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]

    for index, line in enumerate(lines_txt):
        if re.match('(?i:Bachelor)', line):
            bachelor_date_lst = []
            start_idx = index
            for i, l in enumerate(lines_txt[start_idx:start_idx + 8]):
                #print(i, l)
                if datefinder.find_dates(l):
                    matches = datefinder.find_dates(l)
                    for match in matches:
                        #print(match)
                        bachelor_date_lst.append(match)
                    if len(bachelor_date_lst) > 0:
                        return [d.strftime("%m/%Y") for d in bachelor_date_lst]
        elif re.match('BS|B\.S\.|BA|B\.A\.|B\.S\.E|B\.tech' ,line):
            return_set = []
            start_idx = index
            for _, l in enumerate(lines_txt[start_idx:start_idx + 8]):
                if datefinder.find_dates(l):
                    matches = datefinder.find_dates(l)
                    for match in matches:
                        #print(match)
                        return_set.append(match)
                    return [d.strftime("%m/%Y") for d in return_set]

def extract_phd_date(txt1):
    lines_txt = [l.strip() for l in txt1.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    phd_date_lst = []
    for index, line in enumerate(lines_txt):
        if re.match('(Ph\.D|PhD|PhD\.)' ,line):
            start_idx = index
            for i, l in enumerate(lines_txt[start_idx:start_idx + 8]):
                #print(i, l)
                if datefinder.find_dates(l):
                    matches = datefinder.find_dates(l)
                    for match in matches:
                        #print(match)
                        phd_date_lst.append(match)
                    if len(phd_date_lst) > 0:
                        return [d.strftime("%m/%Y") for d in phd_date_lst]

    return phd_date_lst



# --------- aggregate -----------------------------
def get_info(txt):
    tokens_txt = preprocess_text(txt)
    name = get_name(txt, tokens_txt)
    name_ner = get_name_from_ner(txt)
    name_pos = get_name_from_pos(tokens_txt)
    name_ref = get_name_from_ref(tokens_txt)
    # education = extract_education(txt)
    phone_number = extract_phone_number(txt)
    email = extract_email(txt)
    master_degree = extract_master_degree(txt)
    bachelor_degree = extract_bachelor_degree(txt)
    phd_degree = extract_phd_degree(txt)
    master_degree2 = extract_master_degree_from_lines(txt)
    bachelor_degree2 = extract_bachelor_degree_from_lines(txt)
    phd_degree2 = extract_phd_degree_from_lines(txt)

    master_school = extract_ms_school(txt)
    bachelor_school = extract_bach_school(txt)
    phd_school = extract_phd_school(txt)
    master_graduation_date = extract_master_date(txt)
    bachelor_graduation_date = extract_bachelor_date(txt)
    phd_graduation_date = extract_phd_date(txt)
    return {
        'name' : name,
        'name_ner': name_ner,
        'name_pos': name_pos,
        'name_ref': name_ref,
        'phone_number' : phone_number,
        'email' : email,
        'master_degree' : master_degree,
        'bachelor_degree': bachelor_degree,
        'phd_degree' : phd_degree,
        'master_degree2': master_degree2,
        'bachelor_degree2': bachelor_degree2,
        'phd_degree2': phd_degree2,
        'master_school': master_school,
        'bachelor_school': bachelor_school,
        'phd_school': phd_school,
        'master_graduation_date': master_graduation_date,
        'bachelor_graduation_date': bachelor_graduation_date,
        'phd_graduation_date': phd_graduation_date
    }

def parse_resume(path):
    index = os.path.basename(path)
    print(index)
    try:
        raw_txt = read_file(path)
    except pdfminer.pdfparser.PDFSyntaxError:
        return_dict = {'index': index,
                   'name': None,
                   'name_ner': None,
                   'name_pos': None,
                   'name_ref': None,
                   'phone_number': None,
                   'email': None,
                   'master_degree': None,
                   'bachelor_degree': None,
                   'phd_degree': None,
                   'master_degree2': None,
                   'bachelor_degree2': None,
                   'phd_degree2': None,
                   'master_school': None,
                   'bachelor_school': None,
                   'phd_school': None,
                   'master_graduation_date': None,
                   'bachelor_graduation_date': None,
                   'phd_graduation_date': None
                       }
        return return_dict

    if not isinstance(raw_txt, str):
        return_dict = {   'index' : index,
                          'name': None,
                          'name_ner': None,
                          'name_pos': None,
                          'name_ref': None,
                          'phone_number': None,
                          'email': None,
                          'master_degree': None,
                          'bachelor_degree': None,
                          'phd_degree': None,
                          'master_degree2': None,
                          'bachelor_degree2': None,
                          'phd_degree2': None,
                          'master_school': None,
                          'bachelor_school': None,
                          'phd_school': None,
                          'master_graduation_date': None,
                          'bachelor_graduation_date': None,
                          'phd_graduation_date': None}
        return return_dict
    elif len(raw_txt) < 8:
        return_dict = {   'index' : index,
                          'name': None,
                          'name_ner': None,
                          'name_pos': None,
                          'name_ref': None,
                          'phone_number': None,
                          'email': None,
                          'master_degree': None,
                          'bachelor_degree': None,
                          'phd_degree': None,
                          'master_degree2': None,
                          'bachelor_degree2': None,
                          'phd_degree2': None,
                          'master_school': None,
                          'bachelor_school': None,
                          'phd_school': None,
                          'master_graduation_date': None,
                          'bachelor_graduation_date': None,
                          'phd_graduation_date': None}
        return return_dict
    else:
        info_dict = get_info(raw_txt)
        return_dict = {**{'index': index}, **info_dict}
        return return_dict

# -------------- execution ------------------------------------------------------
folder = glob.glob(r"C:/Users/rtd91/Data/resume_samples/*")
# file_path = folder[11]
# sample_txt = read_file(file_path)
# sample_txt2 = read_file2(file_path)
# tokens_txt = preprocess_text(sample_txt)
#
# get_name(sentence_txt=sample_txt, tokens_txt = tokens_txt)
# get_name_from_ner(sample_txt)
# get_name_from_pos(tokens_txt)
# get_name_from_ref(tokens_txt)
# extract_education(sample_txt[0:1000])
# doc = nlp(sample_txt)
# extract_phd_degree(sample_txt)
# extract_phd_degree_from_lines(sample_txt)
# extract_phd_school(sample_txt)
# extract_phd_date(sample_txt)
# extract_master_degree(sample_txt)
# extract_master_degree_from_lines(sample_txt)
# extract_ms_school(sample_txt)
# extract_master_date(sample_txt)
# extract_bachelor_degree(sample_txt)
# extract_bachelor_degree_from_lines(sample_txt)
# extract_bach_school(sample_txt)
# extract_bachelor_date(sample_txt)

df = pd.DataFrame()
for f in folder:
    return_dict = parse_resume(f)
    df = df.append(return_dict, ignore_index = True)

df.to_csv('matched_resumes.csv', index=False)

# # --------- performance ------------------------------
# gt_df = pd.read_csv('./Data/GroundTruth-Sheet2.csv', index_col=False)
# gt_df.shape #182
# gt_df.columns
# gt_df.master_graduation_date
# fuzz.token_sort_ratio("KRISTIN J CHEN", "KRISTIN JIATING CHEN")

721091408_phonescreening.pdf
Bachelor of Engineering in Electronics and Communication Engineering
821101104_phonescreening.pdf


C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:402: DeprecationWarning: Flags not at the start of the expression '(?i)(school|college|' (truncated)


Bachelor of Technology in Mechanical Engineering, Delhi, India       
821101136_phonescreening.pdf
Bachelor of Technology, Instrumentation and Control Engineering
821110120.pdf
Bachelors in Technology, Chemical Engineering with a minor in Pollution Control Engineering
821110440_Mythri.pdf
B S  Chemistry   Minor  Mathematics
821110445_Qizhe.pdf
821110447_rachan.pdf
Bachelor of Technology in Mechanical Engineering    
821110448_RAGHUVEERA.pdf
Bachelor of Technology, Electronics and Communication Engineering, Vellore Institute of Technology
821110464_Saida.pdf
MS in Data Science Graduation     
BS in Mathematics Graduation     
821110509_Akash.pdf
Bachelor of Technology in Chemical Engineering;             
821110510_AKSHARA.pdf
Bachelor of Technology in Computer Science,             
821110511_Alexandra.pdf
Bachelor of Science, Finance     –    
821110527_.pdf
M S , Information Technology and Management
821110528_Wanting.pdf
Bachelor of Arts in Applied Mathematics and Data Science       

KeyboardInterrupt: 

In [21]:
sample_text = "Master of Science Information Systems | 3.54/4 GPA"
def extract_master_degree_from_lines(txt):
    year ="(\d{4})"
    month = "((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?))"
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    master_degrees_lst = []
    ms_lst = []
    for index, line in enumerate(lines_txt):
        if re.match('(?i:Master)' ,line):
            master_degrees_lst.append(line.strip())
        if re.match('MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech' ,line):
            ms_lst.append(line.strip())
    if len(master_degrees_lst) > 0:
        # cleaned_master_degress_lst = []
        # for d in master_degrees_lst:
        #     cleaned_master_degress_lst.append(' '.join([m for m in d.split() if m != month or not m.isdigit()]))
        print(master_degrees_lst)
        mast_deg = [' '.join(m.split()) for m in master_degrees_lst][0]
        print(mast_deg)
        mast_deg1 = re.sub("GPA|.|Current|Cumulative|\d+|–|-|((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?))", " ", mast_deg)
        print(mast_deg1)
        return mast_deg1
    elif len(ms_lst) > 0:
        mast_deg = [' '.join(m.split()) for m in ms_lst][0]
        mast_deg1 = re.sub("GPA|.|Current|Cumulative|\d+|–|-|((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?))", " ", mast_deg)
        print(mast_deg1)
        return mast_deg1
    else:
        mast_deg = None
    return mast_deg

In [22]:
test = extract_master_degree_from_lines(sample_text)
print(test)

['Master of Science Information Systems | 3.54/4 GPA']
Master of Science Information Systems | 3.54/4 GPA
                                                
                                                


In [ ]:
replace_words = ["|","/","."]
sample_txt = "Master of Science Information Systems | 3.54/4 Cumulative Current GPA January 2021" 
sample_txt1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",sample_txt)
sample_txt2 = re.sub("Current|Cumulative|GPA"," ",sample_txt1)
#sample_txt3 = re.sub(r'[\w\s]'," ",sample_txt2)
for punct in replace_words:
    if punct in sample_txt2:
        sample_txt2 = sample_txt2.replace(punct," ")

In [72]:
sample_txt2

'Master of Science Information Systems                  '

In [30]:
def extract_ms_school_from_lines(txt):
    """p = re.compile(
        '(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(M.S.|M.S|M.Sc.|MS|Masters|(?i:Master))')
    
    for m in re.finditer(p, txt):
        mast_school = m.group(0).split("\n")[0]
    """
    ms_school_lst = []
    master_schools_lst = []
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    for index, line in enumerate(lines_txt):
        print(re.findall('(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(?i:Master)' ,line))
        #master_schools_lst.append(line)
        if re.match('(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech)' ,line):
            ms_school_lst.append(line)
    if len(master_schools_lst)>0:
        mast_school = [' '.join(m.split()) for m in master_schools_lst][0]
        mast_school1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",mast_school)
        mast_school2 = re.sub("Current|Cumulative|GPA|Master|of|Science|In|Statistics|â€“|present|Expected|Graduation|Graduating"," ",mast_school1)
        for punct in replace_words:
            if punct in mast_school2:
                mast_school2 = mast_school2.replace(punct," ")
                return mast_school2
    elif len(ms_school_lst)>0:
        mast_school = [' '.join(m.split()) for m in ms_school_lst][0]
        mast_school1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",mast_school)
        mast_school2 = re.sub("Current|Cumulative|GPA|Master|of|Science|In|Statistics|â€“|present|Expected|Graduation|Graduating"," ",mast_school1)
        for punct in replace_words:
            if punct in mast_school2:
                mast_school2 = mast_school2.replace(punct," ")
                return mast_school2



In [28]:
def extract_bach_school_from_lines(txt):
    """p = re.compile(
        '(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(M.S.|M.S|M.Sc.|MS|Masters|(?i:Master))')
    
    for m in re.finditer(p, txt):
        mast_school = m.group(0).split("\n")[0]
    """
    bach_lst = []
    bach_schools_lst = []
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    for index, line in enumerate(lines_txt):
        if re.match('(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(?i:Bachelor)' ,line):    
            bach_schools_lst.append(line)
        if re.match('(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(BS|B\.S\.|BA|B\.A\.|B\.S\.E|B\.tech)' ,line):
            bach_schools_lst.append(line)
    if len(bach_schools_lst)>0:
        bach_school = [' '.join(m.split()) for m in bach_schools_lst][0]
        bach_school1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",bach_school)
        bach_school2 = re.sub("Current|Cumulative|GPA|Master|of|Science|In|Statistics|â€“|present|Expected|Graduation|Graduating"," ",bach_school1)
        for punct in replace_words:
            if punct in bach_school2:
                bach_school2 = bach_school2.replace(punct," ")
                return bach_school2
    elif len(bach_lst)>0:
        bach_school = [' '.join(m.split()) for m in bach_lst][0]
        bach_school1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",bach_school)
        bach_school2 = re.sub("Current|Cumulative|GPA|Master|of|Science|In|Statistics|â€“|present|Expected|Graduation|Graduating"," ",bach_school1)
        for punct in replace_words:
            if punct in bach_school2:
                bach_school2 = bach_school2.replace(punct," ")
                return bach_school2



In [9]:
def extract_phd_school_from_lines(txt):
    """p = re.compile(
        '(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(M.S.|M.S|M.Sc.|MS|Masters|(?i:Master))')
    
    for m in re.finditer(p, txt):
        mast_school = m.group(0).split("\n")[0]
    """
    phd_lst = []
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    for index, line in enumerate(lines_txt):
        if re.match('(?i)(school|college|univers|academy|faculty|institute|faculdades|Schola|schule|lise|lyceum|lycee|polytechnic|kolej|ünivers|okul|University)(.|\n)*(?i:(Ph\.D|PhD|PhD\.))' ,line):    
            phd_lst.append(line)
    if len(phd_lst)>0:
        phd_school = [' '.join(m.split()) for m in phd_lst][0]
        phd_school1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",phd_school)
        phd_school2 = re.sub("Current|Cumulative|GPA|Master|of|Science|In|Statistics|â€“|present|Expected|Graduation|Graduating"," ",phd_school1)
        for punct in replace_words:
            if punct in phd_school2:
                phd_school2 = phd_school2.replace(punct," ")
                return phd_school2



In [7]:
sample_txt = ""' \n\n \n\nTIRTH PATEL \nHolbrook, NY | tirth2410@gmail.com | 469-370-9437 | https://www.linkedin.com/in/tirthpatel24/ \n\nEDUCATION \nThe University of Texas at Dallas, Dallas, TX \nMaster of Science, Business Analytics \nRelevant Coursework:  Statistics and Data Analysis, Data Visualization, Business Analytics with R, Database Foundations, Applied ML \nNirma University, Ahmedabad, India \nBachelor of Technology, Instrumentation and Control Engineering \nStudy Abroad: Graduate Level Summer Course, Studies in Innovation and Entrepreneurship at Harvard Summer School (July 2016) \n\nMay 2021 \nGPA: 3.69 \n\nMay 2018 \n\n \n\nTECHNICAL SKILLS AND CERTIFICATIONS \nDatabases and Languages: \nVisualization Tools: \nAnalysis Tools: \nModeling and Statistical Analysis:  Regression, Classification, Time Series Analysis, Optimization, Hypothesis Testing, A/B Testing, ETL \n\nMySQL, Microsoft SQL Server, MongoDB, Python, R \nTableau, Microsoft Power BI (DAX), Google Data Studio \nSAS, STATA, Google Analytics, Adobe Analytics, MS Excel (Pivot Tables, VLOOKUP, Macros) \n\n \n\nPROFESSIONAL EXPERIENCE \nWeb Analyst, The University of Texas at Dallas | Dallas, TX \n Jan 2021 – May 2021 \n•  Analyzed web traffic behavior for 14 graduate programs using Google Analytics and recommended digital marketing campaigns \n\n \n\nthat increased  monthly active users by 15%. \nImproved data collection efficiencies by troubleshooting roadblocks using Google Tag Manager that enhanced reporting quality. \nIdentified trends and patterns of user behavior to optimize the conversion funnels and reduced exit rates by 10%. \n\n• \n• \n•  Defined and translated KPIs into meaningful insights for 35+ program managers and directors using Data Studio dashboards and \n\n• \n\ntrained them to self-serve their data needs. \nPerformed user navigation  analysis using Google Page Analytics to provide content optimization recommendations, increasing \naverage page value by 20%. \n\nData Analyst, TruSkills Solutions | Ahmedabad, India \n• \n\n May 2018 – Jul 2019 \nFacilitated the development of digital learning solutions by translating product features into functional requirements that allowed \nfor creation and delivery of 1700 hours of interactive content. \n\nStreamlined the process of cleaning and pre-processing Excel files for data import using Python reducing manual efforts by 2x. \nSpearheaded cross-functional planning sessions to propose new product features and improvements based on user feedback. \n\n•  Wrote complex SQL queries to extract data that serve as underpinnings of ad-hoc and recurring analysis. \n•  Developed Tableau dashboards for stakeholders and clients to report user performance and behavior. \n• \n• \nTeaching Assistant, Divyapath Science School | Ahmedabad, India \n•  Delivered supplementary lectures to 150+ high school students to reinforce foundational concepts in Statistics and Calculus. \n•  Held one-on-one mentoring sessions to help students struggling with coursework, achieving a 35% increase in class results. \n• \n\nCollaborated with supervisors to design learning solutions to meet individual student needs, improving course evaluation by 24%. \n\n                              Jan 2017 – May 2018 \n\n \n\n \n\n \n\n \n\nACADEMIC PROJECTS \nEmail Marketing Campaign Analysis (Python – Pandas, Tableau, MS Excel) \nAug 2021 \n•  Analyzed data on 9.5M+ emails sent to over 1M subscribers and their response actions to evaluate the campaign performance. \n•  Discovered trends, patterns and presented recommendations to improve future campaigns and maximize profit from members. \nE-commerce Sales Analysis (Google Analytics, Power BI, MS Excel) \nMay 2020 \n• \nEstablished KPIs for Godiva Chocolatier by analyzing growth and marketing strategies and identified factors affecting the metrics. \n• \nCreated Power BI dashboards for YoY comparison of online sales and presented recommendations to improve performance. \nSocio-Economic Impact of the Opioid Crisis (Tableau, MS Excel, Python – Pandas, Seaborn)                                                        May 2021 \n•  Visualized DEA ARCOS dataset of the distribution of 500M opioid pills from 2008-2014 across the US to find insights and trends. \n• \nPredicting Responses to Marketing Campaign (Python – NumPy, Scikit-Learn, Pandas, Matplotlib) \n• \n• \nUS Wildfires Data Analysis (MySQL, MongoDB)                                                                                                                                         Oct 2019 \n•  Modeled a relational database from raw dataset of 1.88 million US wildfires and implemented the normalized database in MySQL. \n• \nPerformed data extraction and analysis to discover hidden insights using joins, nested conditions, and complex SQL queries. \nPrediction of Credit Card Defaulters (MS Excel, R - dplyr, ggplot2, shiny) \n•  Analyzed historical data of 30K credit card clients to determine most important factors resulting in default payments.  \n• \n\nBuilt statistical models using ML algorithms for target variable prediction and improved accuracies by tuning hyperparameters. \nPredicted with 86% accuracy if a customer will subscribe to term deposit in response to marketing campaign with kernelized SVM. \n\nEmployed various data mining techniques to analyze the impact on highly affected counties and provided measures for mitigation. \n\nImplemented various algorithms such as logistic regression, decision trees and neural networks to build a predictive model that \nidentifies potential defaulters with an accuracy of 82%.  \n\n  Dec 2020  \n\nFeb 2020 \n\n \n\n \n\n \n\n \n\n \n\n\x0c'""
sample_txt1 = "'SAIDA MUKTAR \n\nPhone: 443-833-6344 \n\nEmail: saidam1@umbc.edu \n\nEDUCATION \nMS in Data Science     Graduation: August 2021 \nGraduate Certificate in Cybersecurity Operations  \nRelevant courses: Introduction to Data Science, Introduction to Data Analysis and Machine Learning, Systems and Information Integration, Ethical and \nLegal Issues in Data Science, Data Management, Big Data Platforms, Introduction to Cybersecurity, Risk Analysis & Compliance, Managing \nCyberSecurity Operations  \n \nBS in Mathematics      Graduation: December 2019  \nRelevant courses: Calculus I/II/III, Computer Science (Python), Linear Algebra, Differential Equations, Partial Differential Equations, Introduction to \nMath Analysis I/II, Physics I/II, Biomathematics, Dynamical Systems, Mathematical Physiology \n \nTECHNICAL PROFILE \nOperating systems: Unix(Linux), Windows  \nTechnologies: SQL, Python, R, MATLAB, Maple, LaTeX, MySQL, Spark, Azure, Hadoop, MapReduce, Databricks, Pandas, Numpy, Scikit-learn, Seaborn, \nMatplotlib, Folium, PyTorch, Tensoflow \nData Science Skills: Statistical analysis, Predictive modeling, simulation, Deep Learning models, Time-Series analysis, Clustering, Decision Tree Learning, \nArtificial Neural Networks, Regression, Exploratory Data Analysis, Data Engineering, Data Prep, Traditional Predictive Modeling, Statistical Analysis, \nNatural Language Processing  \n\n \n\nEXPERIENCE \nONCOSPACE         \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n  2020 – PRESENT  \n\n• \n\nDesigned models using both supervised and unsupervised machine learning methods that predict radiation doses for oncologists  \n\n- \n\n- \n\nCreated a report of validation accuracy metrics (R^2, mean absolute error, area between the curve, shortest distance error) for \neach grouping of plans and identification of plans for which the auto planning results are more or less accurate than is typical  \nPrepared a report of any significant error patterns identified in the auto planning model results and characteristics of plans which \nexhibited each error pattern \n- \nDVH clustering for refraction of plan model assignment  \n-  Made use of KMean clustering for machine learning analysis  \n- \n- \n\nOperated different graphing techniques in order to check what variables in our model were causing the clusters. \nImplemented Principal Component Analysis (PCA) in order to check correlation between clusters using OVH graph \nAnalyzed a variety of large and complex data sets using a variety of standard analytic techniques including regression, and machine \nlearning approaches \nUsed data from SQL databases in python to study information of patients \n\n• \n•       Presented and summarized analyses in various formats including raw output, tables, and graphics, oral and written reports \n• \n• \n• \n•  Wrote manual test cases using Azure DevOps  \n\nDesigned tables and graphs for visualization using Pandas and Matplotlib \nUtilized statistical packages in order to analyze results from Machine Learning outputs \nFamiliarized with Azure Cloud as we utilize it as the primary platform for most data studies \n\n• \n\nUNIVERSITY OF MARYLAND BALTIMORE COUNTY      \n\n 2017  – 2019 \n\nStudied the time course of nuclear-cytoplasmic distribution of Foxo1 \n\n• \n•  Made mathematical models to show the modulation of nuclear influx and efflux of Foxo1 by IGF-I/PI3K/Akt pathway in skeletal muscle \n\n•  Used mathematical models of nuclear-cytoplasmic movements of Foxo1 to provide values of unidirectional influx and efflux under various \n\n•  Made mathematical models to help determine the properties of Foxo1 phosphorylation/dephosphorylation status in nuclei and cytoplasm \n\nfibers. \n\nexperimental conditions \n\nof skeletal muscle fibers. \n\n•  Made two-state mathematical models to study the Foxo1 nuclear-cytoplasmic movements \n•  Used MATLAB to test the models by fitting date from blocking nuclear efflux with Leptomycin \n• \n•  Used LaTex for project reporting \n\nPlotted datas in MATLAB to find parameters and fit parameters \n\nUNIVERSITY OF MARYLAND BALTIMORE  \n\n \n\n \n\n 2020 \n\nStudying levels of serotonin release in brain when pain is inflicted \n\n• \n•  Making Python and Matlab codes that will record and analyze data when pain is recorded after a given threshold \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \nREFERENCE AVAILABLE UPON REQUEST \n\nBS degree in statistics , ma chine lear ning, or data s cien ce  \nBS degree, or e quivalent experien ce, in syste m engi neering or  decisi on the ory \nTwo or more years’ experien ce desig ning and leading 50 -per son  or larger techni cal progra ms  \nEvidence  of sel f-directe d organizational investigation, diag nosis , and re development  \nTwo or more years’ experien ce usi ng simulation or ot her analytical tools to  model and  control software developme nt  \nPublicati on hist ory that demonstrates e ffe ctive technical communication skills  \nPreferred Qualifications  \n \nMS degree in statistics,  machine learni ng, or data scie nce  \nMS or PhD degree i n system e ngineering  or deci sion theory  \nTwo or more years’ experien ce desig ning, leadi ng, and  optimizi ng 100 -pers on  or larger techni cal progra ms.  \nFive or more years’ experien ce de signing and buil ding si mulations or other a nalytical tools to  model a nd control s oftware development  \nPublicati on hist ory that demonstrates e ffe ctive technical communication skills. Speci fically, publications that docume nts sel f-dire cted orga nizational inve stigation, diagnosis, and redevelopment recomme ndations.  \nSpace industry experien ce in the private, military, or civilian se ctor.  \n \n\n\x0c'"

In [31]:
extract_ms_school_from_lines(sample_txt1)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [19]:
extract_bach_school_from_lines(sample_txt1)

In [20]:
extract_phd_school_from_lines(sample_txt1)

In [ ]:
def extract_master_degree_from_lines(txt):
    year ="(\d{4})"
    month = "((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?))"
    lines_txt = [l.strip() for l in txt.split('\n') if len(l.strip()) > 0 and len(l.strip().split()) > 0]
    master_degrees_lst = []
    ms_lst = []
    for index, line in enumerate(lines_txt):
        if re.match('(?i:Master)' ,line):
            master_degrees_lst.append(line.strip())
        if re.match('MS|M\.S\.|MA|M\.A\.|MBA|M\.S\.E|M\.tech' ,line):
            ms_lst.append(line.strip())
    if len(master_degrees_lst) > 0:
        # cleaned_master_degress_lst = []
        # for d in master_degrees_lst:
        #     cleaned_master_degress_lst.append(' '.join([m for m in d.split() if m != month or not m.isdigit()]))
        mast_deg = [' '.join(m.split()) for m in master_degrees_lst][0]
        mast_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",mast_deg)
        mast_deg2 = re.sub("Current|Cumulative|GPA"," ",mast_deg1)
        for punct in replace_words:
            if punct in mast_deg2:
                mast_deg2 = mast_deg2.replace(punct," ")
        return mast_deg2
    elif len(ms_lst) > 0:
        mast_deg = [' '.join(m.split()) for m in ms_lst][0]
        mast_deg1 = re.sub("\d+|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)", " ",mast_deg)
        mast_deg2 = re.sub("Current|Cumulative|GPA"," ",mast_deg1)
        for punct in replace_words:
            if punct in mast_deg2:
                mast_deg2 = mast_deg2.replace(punct," ")
        print(mast_deg2)
        return mast_deg2
    else:
        mast_deg2 = None
        return mast_deg2